# __Segmenting and Clustering Neighbourhoods in Toronto__
This notebooks is used for the Week 3 assignment within the Courser course 'Applied Data Science Capstone'

## Part 1
We start with extracting the data from the wikipedia page <br/>
To replace 'Not assigned' with NaN we set up the parameter 'na_values'. That will help us to modify the dataset later.

In [1]:
# Let's start with extracting the data from the Wikipedia page

import pandas as pd
import numpy as np

# We set the parameter 'na_values' so to replase the values 'Not assigned' with NaN
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', na_values = 'Not assigned')[0]
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Now we have to format and prepare data. To do it:
> We remove all the rows where 'Borough' is NaN <br/>
> We replace NaN in 'Neighbourhoods' column with values from 'Borough' one

In [2]:
# Let's remove all the rows where 'Borough' is NaN
data.dropna(axis=0, subset=['Borough'], inplace=True)

# Let's replace NaN in Neighbourhoods with Borough values
data.fillna(axis=1, method='ffill', inplace=True)

Now we have to combine rows with the same Postocode and aggregate their Neighbourhood with comma separator

In [3]:
data1=data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join)

To present the results as DataFrame of a proper format we will reset both levels of the index. <br/>
To be shure that everything is OK let's look at the first 30 rows and look for the indexes on the screenshot provided as an example.

In [4]:
data1=pd.DataFrame(data1)
data1.reset_index(level=1, inplace=True)
data1.reset_index(level=0, inplace=True)

# Let's look at the first 30 rows and compare some of them with the screenshot provided
data1.head(30)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


And finally we have to print the number of the rows with .shape method

In [5]:
print("The number of rows of the final dataframe: ", data1.shape[0])

The number of rows of the final dataframe:  103


## Part 2
Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhoods name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

Let's initialize the arrays for latitude and longitude information.

In [6]:
m = data1.shape[0]
latitude = [0] * m
longitude = [0] * m

First, we will try to get the longitude coordinates via geocoder, though that package can be very unreliable and has the limit of 2500 calls daily.<br/>
For beginning we have to install geocoder.

In [7]:
!conda install -c conda-forge geocoder --yes # uncomment this line if you run the cell for the first time or in case of error message "Name '...' is not defined"
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [ ]:
p_codes=data1['Postcode']
for i in range(0,(m-1)):
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # pick up the postal_code
    postal_code = p_codes[i]
  
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        if lat_lng_coords != None:
            latitude[i] = lat_lng_coords[0]
            longitude[i] = lat_lng_coords[1]
geo_coord = [latitude logitude]

If we failed to get the information via geocoder, we will use the alternative way and download the file with the geo coordinates. <br/>
To check geocoder is a success we'll look at the latiture for the last postcode. Its value '0' indicates that geocoder failed.

In [8]:
#Let's see whether geocoder has changed the latitude for the last postcode. If not, we'll download the geo coordinates from the link provided
if latitude[m-1] == 0:
    print("Since geocoder failed, data will be loaded from the file")
    !wget -q -O 'geo_coordinates.csv' https://cocl.us/Geospatial_data
    print('Data downloaded!')
    geo_coord = pd.read_csv('geo_coordinates.csv')

Since geocoder failed, data will be loaded from the file
Data downloaded!


In [9]:
geo_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we have to add the geo coordinates to our data frame.<br/>
Both the frames have Postcode columns (but under different names), so we have to merge dataframes accordingly the information in that column.

In [10]:
# First let's correct the name of the column in geo_coord frame
geo_coord.rename(columns={'Postal Code':'Postcode'}, inplace=True)

# let's merge the frames by values in the Postcode column
data2 = pd.merge(data1, geo_coord, on='Postcode')

data2.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3
Now we are going to explore and cluster the neighborhoods in Toronto. <br/>
### It is strongly recommended to run this code in Chrome since most probably you will not see the maps in Edge or Explorer <br/>
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Let's start with installing the libraries we'll need

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you run the cell for the first time or in case of error message "Name '...' is not defined"
import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you run the cell for the first time in case of error message "Name '...' is not defined"
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [12]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Let's find the geographical coordinates of Toronto

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
tor_location = geolocator.geocode(address)

tor_latitude = tor_location.latitude
tor_longitude =tor_location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_latitude, tor_longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Now we will create map of Toronto using latitude and longitude values and add markers to map.

In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(data2['Latitude'], data2['Longitude'], data2['Borough'], data2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Downtown where Toronto Univercity is placed. So let's slice the original dataframe and create a new dataframe of the Downtown data.

In [15]:
DT_data = data2[data2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


Let's get the geographical coordinates of Downtown.

In [16]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown are 43.655115, -79.380219.


As we did with all of Toronto, let's visualize Downtown the neighborhoods in it.

In [17]:
# create map of Downtown using latitude and longitude values
map_DT = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'VC5VAI2CNSBEQ1BIWEREK0RDJX2VK4WVRYEXQTSLB4Q4XXFB' #  Foursquare ID
CLIENT_SECRET = 'HRGWOCZXY555UGMEUD4APX5LYXYTREVQNOLEQZJT1YFRW3U3' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version. In fact, I didn't find the information about it on Foursquare site

print('The credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

The credentails:
CLIENT_ID: VC5VAI2CNSBEQ1BIWEREK0RDJX2VK4WVRYEXQTSLB4Q4XXFB
CLIENT_SECRET:HRGWOCZXY555UGMEUD4APX5LYXYTREVQNOLEQZJT1YFRW3U3


#### Let's explore the first neighbourhood in our dataframe.

Get the neighbourhood's name.

In [19]:
DT_data.loc[0, 'Neighbourhood']

'Rosedale'

Get the neighbourhood's latitude and longitude values.

In [20]:
neighbourhood_latitude = DT_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = DT_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = DT_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


#### Now, let's get the top 100 venues that are in Rosedale within a radius of 1000 meters.

First, let's create the GET request URL. Name your URL **url**.

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VC5VAI2CNSBEQ1BIWEREK0RDJX2VK4WVRYEXQTSLB4Q4XXFB&client_secret=HRGWOCZXY555UGMEUD4APX5LYXYTREVQNOLEQZJT1YFRW3U3&v=20180605&ll=43.6795626,-79.37752940000001&radius=1000&limit=100'

Send the GET request and get the resutls

In [22]:
results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
2,Black Camel,BBQ Joint,43.677016,-79.389367
3,Tinuno,Filipino Restaurant,43.671281,-79.374920
4,Craigleigh Gardens,Park,43.678099,-79.371586
5,Maison Selby,Bistro,43.671232,-79.376618
6,Pie Squared,Pie Shop,43.672143,-79.377856
7,Starbucks,Coffee Shop,43.671082,-79.380756
8,Manulife Financial,Office,43.672070,-79.382449
9,Booster Juice,Smoothie Shop,43.671566,-79.378581


nd how many venues were returned by Foursquare?

In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


## OK. Now let's explore Neighbourhoods in Downtown

#### Let's create a function to repeat the same process to all the neighborhoods in Downtown

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighbourhood and create a new dataframe called *DT_venues*.

In [27]:
DT_venues = getNearbyVenues(names=DT_data['Neighbourhood'],
                                   latitudes=DT_data['Latitude'],
                                   longitudes=DT_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


#### Let's check the size of the resulting dataframe

In [28]:
print(DT_venues.shape)
DT_venues.head()

(1291, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


Let's check how many venues were returned for each neighborhood <br/>
***Remember that we set up LIMIT that is the max number of results as 100***

In [29]:
DT_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,90,90,90,90,90,90
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(DT_venues['Venue Category'].unique())))

There are 209 uniques categories.


## Now let's analyze each neighborhood

In [31]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
DT_onehot['Neighbourhood'] = DT_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [32]:
DT_onehot.shape

(1291, 210)

#### Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [33]:
DT_grouped = DT_onehot.groupby('Neighbourhood').mean().reset_index()
DT_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.000000,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.0,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011905,0.000000,...,0.000000,0.00,0.0,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.01,0.0,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000
6,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011111,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011111,0.000000,...,0.011111,0.00,0.0,0.00,0.000000,0.011111,0.011111,0.000000,0.011111,0.011111
8,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.000000,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


#### Let's confirm the new size

In [34]:
DT_grouped.shape

(18, 210)

#### Let's print each neighbourhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in DT_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Steakhouse  0.04
3          Café  0.04
4   Cheese Shop  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4  Sculpture Garden  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2                Café  0.07
3  Italian Restaurant  0.04
4              Bakery  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.05
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4      Ice Crea

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighbourhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = DT_grouped['Neighbourhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Cosmetics Shop,Restaurant,Thai Restaurant,American Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Bakery,Café,French Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Pizza Place,Outdoor Sculpture,Park,Breakfast Spot
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant


## Let's cluster neighbourhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [38]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 4, 2, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DT_merged = DT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
DT_merged = DT_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

DT_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Building,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Pizza Place,Outdoor Sculpture,Park,Breakfast Spot
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Gym,Pub,Fast Food Restaurant
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Park,Mexican Restaurant,Gym / Fitness Center,Breakfast Spot,Pub,Theater,Cosmetics Shop
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Diner,Restaurant,Bakery,Plaza


Finally, let's visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighbourhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finally let's examine clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

#### Cluster 1

In [41]:
DT_merged.loc[DT_merged['Cluster Labels'] == 0, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Pizza Place,Outdoor Sculpture,Park,Breakfast Spot
2,Downtown Toronto,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Gym,Pub,Fast Food Restaurant
3,Downtown Toronto,0,Coffee Shop,Bakery,Café,Park,Mexican Restaurant,Gym / Fitness Center,Breakfast Spot,Pub,Theater,Cosmetics Shop
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Diner,Restaurant,Bakery,Plaza
5,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Clothing Store,Beer Bar,Gastropub,Cosmetics Shop,Bakery
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Bakery,Café,French Restaurant
7,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Japanese Restaurant,Chinese Restaurant,Sushi Restaurant
8,Downtown Toronto,0,Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Cosmetics Shop,Restaurant,Thai Restaurant,American Restaurant,Hotel
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Fried Chicken Joint,Brewery,Sports Bar,Sporting Goods Shop,Bakery
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Bar,Gastropub,Bakery,Italian Restaurant,American Restaurant


#### Cluster 2

In [42]:
DT_merged.loc[DT_merged['Cluster Labels'] == 1, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Building,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 3

In [43]:
DT_merged.loc[DT_merged['Cluster Labels'] == 2, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Athletics & Sports,Diner,Nightclub,Convenience Store,Coffee Shop


#### Cluster 4

In [44]:
DT_merged.loc[DT_merged['Cluster Labels'] == 3, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden


#### Cluster 5

In [45]:
DT_merged.loc[DT_merged['Cluster Labels'] == 4, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,4,Café,Japanese Restaurant,Bar,Bakery,Restaurant,Bookstore,Noodle House,Chinese Restaurant,Italian Restaurant,Dessert Shop
13,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Bakery,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Coffee Shop,Dessert Shop


Now we can assign new names to clusters and display the map again.

In [46]:
new_labels = ['Coffee shops', 'Parks', 'Grocery Stores', 'Airport Lounges', 'Cafe']

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighbourhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(new_labels[cluster]), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters